# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
def download_data(url_zip, downloadpath, file):
    """Download and extract the training data."""
    import urllib
    from zipfile import ZipFile
    data_file = f'./{downloadpath}/archive.zip'
    download_url = 
    urllib.request.urlretrieve(download_url, filename=data_file)

    # extract files
    with ZipFile(data_file, 'r') as zip:
        print('extracting files...')
        zip.extractall()
        print('finished extracting')
        data_dir = zip.namelist()[0]

    # delete zip file
    os.remove(data_file)
    
    
    return data_dir

ws = Workspace.from_config()
zipurl = 'https://www.kaggle.com/rupeshraundal/marketcheck-automotive-data-us-canada/download'
filenm = 'ca-dealers-used.csv'
downloadpath = 'Data'
dskey = 'UsedCarPrices'
dsdesc = 'Used Car Price Training Data'
found = False

if key in ws.datasets.keys(): 
    found = True
    dataset = ws.datasets[key] 

if not found:
    datapath = download_data(zipurl, downloadpath, filenm)

    # get the datastore to upload prepared data
    datastore = ws.get_default_datastore()

    # upload the local file from src_dir to the target_path in datastore
    datastore.upload(src_dir=downloadpath, target_path=downloadpath)
    dataset = Dataset.Tabular.from_delimited_files(path=[(datastore, (os.join(downloadpath, filenm)))], separator=',', header=True, encoding='utf8')
    dataset = dataset.register(workspace=workspace,
                                 name=dskey,
                                 description=dsdesc)
    # dataset = Dataset.Tabular.from_delimited_files(os.join(datapath, filenm), separator=',', header=True, encoding='utf8')
    # https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.tabulardataset?view=azure-ml-py
    # https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-register-datasets#explore-data
    # https://docs.microsoft.com/en-us/azure/machine-learning/how-to-train-with-datasets
    
#########################################################


experiment_name = 'PCAP_Exp_CarPrices_AutoML'
experiment=Experiment(ws, experiment_name)


amlcompute_cluster_name = "PCAP-Clus-DS12v2"
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 2)


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-automlstep-in-pipelines
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 60,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'r2_score',
    "verbosity": logging.INFO,
    "n_cross_validations": 5
}
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train
# Metrics like r2_score and spearman_correlation can better represent the quality of model when the scale of the value-to-predict covers many orders of magnitude. 
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train
# TODO: Put your automl config here
automl_config = AutoMLConfig(
    compute_target=compute_target,
    task = "regression",
    training_data=dataset,
    label_column_name="price",  
    path = project_folder,
    enable_early_stopping= True,
    featurization= 'auto',
    debug_log = "automl_errors.log",
    **automl_settings
)

In [ ]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

automl_step = AutoMLStep(
    name='automl_module_train',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_automl_train",
    workspace=ws,    
    steps=[automl_step])

# TODO: Submit your experiment
pipeline_run = experiment.submit(pipeline)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

pipeline_run.wait_for_completion()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
# https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/automated-machine-learning/forecasting-energy-demand/auto-ml-forecasting-energy-demand.ipynb
#todo CONSIDER THIS ALTERNATE OF RUNNING AUTOML WITHOUT A PIPELINE.

metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
print(df)

from azureml.train.automl.run import AutoMLRun
#from azureml.widgets import RunDetails

# workaround to get the automl run as its the last step in the pipeline 
# and get_steps() returns the steps from latest to first

for step in pipeline_run.get_steps():
    automl_step_run_id = step.id
    print(step.name)
    print(automl_step_run_id)
    break

automl_run = AutoMLRun(experiment = experiment, run_id=automl_step_run_id)

# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train
# best_run, fitted_model = automl_run.get_output()
# print(best_run)
# print(fitted_model)
# https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/machine-learning-pipelines/nyc-taxi-data-regression-model-building/nyc-taxi-data-regression-model-building.ipynb
# https://docs.microsoft.com/en-us/azure/machine-learning/tutorial-auto-train-models

In [ ]:
#TODO: Save the best model
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
print(best_model)

print(best_model.steps)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
myenv_inference = Environment(workspace=ws, name="AzureML-sklearn-0.24.1-ubuntu18.04-py37-cpu-inference") 

model = best_run.register_model(description ='AutoML-AutoPrices')

print(f"Model ID - {best_run.model_id}")

# Combine scoring script & environment in Inference configuration
dummy_inference_config = InferenceConfig(entry_script="score.py", environment=myenv_inference)

# Set deployment configuration
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, auth_enabled=True)

ws_name = "SVC_AutoPrices"
service = Model.deploy(
    ws,
    ws_name,
    [model],
    inference_config = dummy_inference_config,
    deployment_config = deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)
print(service.state)
print(service.scoring_uri)
print(service.swagger_uri)
primary, secondary = service.get_keys()
print(primary)
#Enable app insights
service.update(enable_app_insights=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:

service = Webservice(workspace=ws, name=ws_name)


import requests
# URL for the web service
scoring_uri = service.scoring_uri
# If the service is authenticated, set the key or token
key = primary

# Two sets of data to score, so we get two results back

#Generally, I figure we would send in all the columns from the original dataset and 
#then run it through the clean function to whittle it down, but that's not necessary for this exercise.
data = {
    "data":
    [
        {
            "id":"2fc65749-0cff",
            "vin":"1FTEW1E50JFB75149",
            "miles":"51611",
            "stock_no":"2021128A",
            "year":"2018",
            "make":"Ford",
            "model":"F-150",
            "trim":"XLT",
            "body_type":"Pickup",
            "vehicle_type":"Truck",
            "drivetrain":"4WD",
            "transmission":"Automatic",
            "fuel_type":"Unleaded / Unleaded",
            "engine_size":"5",
            "engine_block":"V",
            "seller_name":"holiday ford lincoln ltd",
            "street":"1555 Lansdowne Street, Peterborough",
            "city":"Peterborough",
            "state":"ON",
            "zip":"K9J 7M3"

            #Should be around 44699
        }
    ,
        {
            "id":"cd8fc59c-4ff5",
            "vin":"2T1BE40E39C017603",
            "miles":"335361",
            "stock_no":"1385",
            "year":"2009",
            "make":"Toyota",
            "model":"Corolla",
            "trim":"XRS",
            "body_type":"Sedan",
            "vehicle_type":"Car",
            "drivetrain":"FWD",
            "transmission":"Manual",
            "fuel_type":"Unleaded",
            "engine_size":"2.4",
            "engine_block":"I",
            "seller_name":"fara auto sales",
            "street":"1545 Warden Ave",
            "city":"Scarborough",
            "state":"ON",
            "zip":"M1R 2S9"

            #Should be around 3550
        }
    ,
        {
            "id":"77521bd9-e533",
            "vin":"W1N0G6EB4LV247708",
            "miles":"8852",
            "stock_no":"M21144A",
            "year":"2020",
            "make":"Mercedes-Benz",
            "model":"GLC-Class",
            "trim":"AMG GLC43",
            "body_type":"SUV",
            "vehicle_type":"Truck",
            "drivetrain":"4WD",
            "transmission":"Automatic",
            "fuel_type":"Premium Unleaded",
            "engine_size":"3",
            "engine_block":"V",
            "seller_name":"kelowna mercedes-benz",
            "street":"2580 Enterprise Way",
            "city":"Kelowna",
            "state":"BC",
            "zip":"V1X 7X5"

            #Should be around 79995
        }
    ]
}
# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-log-view-metrics

print(service.get_logs())

service.delete()